## 卷积神经网络（LeNet）
在$3.9$节（多层感知机的从零开始实现）里我们构造了一个含单隐藏层的多层感知机模型来对$Fashion-MNIST$数据集中的图像进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为$784$的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性。

1. 图像在**同一列**邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于大尺寸的输入图像，使用全连接层容易造成模型过大。假设输入是高和宽均为$1000$像素的彩色照片（含$3$个通道）。即使全连接层输出个数仍是$256$，该层权重参数的形状是$3,000,000\times 256$：它占用了大约$3$ $GB$的内存或显存。这带来过**复杂的模型和过高的存储开销**。

卷积层尝试解决这两个问题。一方面，**卷积层保留输入形状**，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，**卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算**，从而避免参数尺寸过大。

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：$LeNet$。

### LeNet模型
$LeNet$分为**卷积层块**和**全连接层块**两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是**卷积层后接最大池化层**：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的**最大池化层则用来降低卷积层对位置的敏感性**。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用$sigmoid$激活函数。第一个卷积层输出通道数为$6$，第二个卷积层输出通道数则增加到$16$。这是因为**第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似**。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为$2$。由于池化窗口与步幅形状相同，**池化窗口在输入上每次滑动所覆盖的区域互不重叠**。

卷积层块的输出形状为 **(批量大小, 通道, 高, 宽)**。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平$（flatten）$。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且**向量长度为通道、高和宽的乘积**。全连接层块含$3$个全连接层。它们的输出个数分别是$120$、$84$和$10$，其中$10$为输出的类别个数。

下面我们通过$Sequential$类来实现$LeNet$模型。

In [1]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
from d2lzh_pytorch import evaluate_accuracy
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print(device)
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
        
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

cuda:7


接下来查看每个层的形状。

In [2]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用**高和宽均为$5$的卷积核，从而将高和宽分别减小$4$，而池化层则将高和宽减半**，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。

### 获取数据和训练模型
下面我们来实验$LeNet$模型。实验中，我们仍然使用$Fashion-MNIST$作为训练数据集。

In [3]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size, root='~/wms/Jupyter/fyc/Datasets/FashionMNIST')

In [4]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs):
    net = net.to(device)
    print("training on:",device)
    loss = torch.nn.CrossEntropyLoss()
    for epoch in range(1,epochs+1):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1)==y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net, device=device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

学习率采用$0.001$，训练算法使用$Adam$算法，损失函数使用交叉熵损失函数。

In [5]:
lr, epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, epochs)

training on: cuda:7
step 1, train_acc: 0.1016
step 2, train_acc: 0.1133
step 3, train_acc: 0.1133
step 4, train_acc: 0.1035
step 5, train_acc: 0.1031
step 6, train_acc: 0.0964
step 7, train_acc: 0.0960
step 8, train_acc: 0.1035
step 9, train_acc: 0.1011
step 10, train_acc: 0.1020
step 11, train_acc: 0.1023
step 12, train_acc: 0.1016
step 13, train_acc: 0.1037
step 14, train_acc: 0.1038
step 15, train_acc: 0.1021
step 16, train_acc: 0.1028
step 17, train_acc: 0.1059
step 18, train_acc: 0.1057
step 19, train_acc: 0.1057
step 20, train_acc: 0.1049
step 21, train_acc: 0.1045
step 22, train_acc: 0.1053
step 23, train_acc: 0.1050
step 24, train_acc: 0.1056
step 25, train_acc: 0.1052
step 26, train_acc: 0.1050
step 27, train_acc: 0.1039
step 28, train_acc: 0.1042
step 29, train_acc: 0.1045
step 30, train_acc: 0.1048
step 31, train_acc: 0.1043
step 32, train_acc: 0.1041
step 33, train_acc: 0.1040
step 34, train_acc: 0.1035
step 35, train_acc: 0.1066
step 36, train_acc: 0.1075
step 37, train_ac

step 74, train_acc: 0.5979
step 75, train_acc: 0.5976
step 76, train_acc: 0.5976
step 77, train_acc: 0.5974
step 78, train_acc: 0.5970
step 79, train_acc: 0.5965
step 80, train_acc: 0.5965
step 81, train_acc: 0.5966
step 82, train_acc: 0.5965
step 83, train_acc: 0.5960
step 84, train_acc: 0.5960
step 85, train_acc: 0.5966
step 86, train_acc: 0.5961
step 87, train_acc: 0.5959
step 88, train_acc: 0.5956
step 89, train_acc: 0.5963
step 90, train_acc: 0.5966
step 91, train_acc: 0.5968
step 92, train_acc: 0.5970
step 93, train_acc: 0.5978
step 94, train_acc: 0.5980
step 95, train_acc: 0.5982
step 96, train_acc: 0.5981
step 97, train_acc: 0.5987
step 98, train_acc: 0.5989
step 99, train_acc: 0.5995
step 100, train_acc: 0.5997
step 101, train_acc: 0.5996
step 102, train_acc: 0.5998
step 103, train_acc: 0.5997
step 104, train_acc: 0.5998
step 105, train_acc: 0.5996
step 106, train_acc: 0.5995
step 107, train_acc: 0.6001
step 108, train_acc: 0.6000
step 109, train_acc: 0.6001
step 110, train_ac

step 148, train_acc: 0.7055
step 149, train_acc: 0.7053
step 150, train_acc: 0.7057
step 151, train_acc: 0.7057
step 152, train_acc: 0.7060
step 153, train_acc: 0.7060
step 154, train_acc: 0.7058
step 155, train_acc: 0.7058
step 156, train_acc: 0.7059
step 157, train_acc: 0.7061
step 158, train_acc: 0.7062
step 159, train_acc: 0.7065
step 160, train_acc: 0.7066
step 161, train_acc: 0.7064
step 162, train_acc: 0.7063
step 163, train_acc: 0.7064
step 164, train_acc: 0.7063
step 165, train_acc: 0.7062
step 166, train_acc: 0.7063
step 167, train_acc: 0.7064
step 168, train_acc: 0.7064
step 169, train_acc: 0.7065
step 170, train_acc: 0.7068
step 171, train_acc: 0.7067
step 172, train_acc: 0.7070
step 173, train_acc: 0.7073
step 174, train_acc: 0.7072
step 175, train_acc: 0.7073
step 176, train_acc: 0.7076
step 177, train_acc: 0.7075
step 178, train_acc: 0.7075
step 179, train_acc: 0.7079
step 180, train_acc: 0.7081
step 181, train_acc: 0.7080
step 182, train_acc: 0.7080
step 183, train_acc:

step 209, train_acc: 0.7395
step 210, train_acc: 0.7393
step 211, train_acc: 0.7394
step 212, train_acc: 0.7393
step 213, train_acc: 0.7391
step 214, train_acc: 0.7392
step 215, train_acc: 0.7393
step 216, train_acc: 0.7395
step 217, train_acc: 0.7395
step 218, train_acc: 0.7396
step 219, train_acc: 0.7399
step 220, train_acc: 0.7400
step 221, train_acc: 0.7400
step 222, train_acc: 0.7399
step 223, train_acc: 0.7397
step 224, train_acc: 0.7398
step 225, train_acc: 0.7398
step 226, train_acc: 0.7400
step 227, train_acc: 0.7400
step 228, train_acc: 0.7400
step 229, train_acc: 0.7402
step 230, train_acc: 0.7401
step 231, train_acc: 0.7402
step 232, train_acc: 0.7400
step 233, train_acc: 0.7398
step 234, train_acc: 0.7399
step 235, train_acc: 0.7399
epoch 4, loss 0.6774, train acc 0.740, test acc 0.746, time 2.3 sec
step 1, train_acc: 0.7148
step 2, train_acc: 0.7441
step 3, train_acc: 0.7526
step 4, train_acc: 0.7451
step 5, train_acc: 0.7523
step 6, train_acc: 0.7598
step 7, train_acc: 0

+ 卷积神经网络就是含卷积层的网络。
+ LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。